In [ ]:
#### import packages and housekeep ####

In [3]:
import pandas as pd
import numpy as np
import datetime
import statsmodels.api as sm
import os
import sys
import logging
from Matcher import Matcher
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, KFold
import matplotlib.pyplot as plt
## Setup code every *_task.py file needs
# piper_path = os.path.abspath(__file__)
# while os.path.basename(piper_path) != 'piper':
#     piper_path = os.path.dirname(piper_path)
# if piper_path not in sys.path:
#     sys.path.append(piper_path)
# from piper.piper_utils import setup_piper
# setup_piper()

# from piper.db.snowflake import SnowflakeDb
# db = SnowflakeDb.singleton()

In [4]:
import seaborn as sns

In [5]:
import instaquery as iq

In [6]:
pd.set_option('display.max_columns', None)

In [7]:
from piper.db.snowflake import SnowflakeDb

In [8]:
from datetime import date
from datetime import timedelta

In [9]:
import jinja2

In [ ]:
#### generate attributes ####

In [10]:
def date_ref_table_gen (start_date, cohort_duration_days, num_cohorts, date_ref_df = pd.DataFrame(), date_ref_temp_df = pd.DataFrame()): #start_date_inclusive 
    for n in range(num_cohorts):
#         print(n)
        
        start_date_temp = (date.fromisoformat(start_date) + timedelta(days=(n*cohort_duration_days))).isoformat()
        visit_start_temp = (date.fromisoformat(start_date_temp) - timedelta(days=(cohort_duration_days))).isoformat()
        end_date_temp = (date.fromisoformat(start_date_temp) + timedelta(days=(cohort_duration_days-1))).isoformat()

#         print(start_date_temp)
        date_ref_temp_df.loc[n, 'visit_start'] = visit_start_temp
        date_ref_temp_df.loc[n, 'start_date'] = start_date_temp 
        date_ref_temp_df.loc[n,'end_date'] = end_date_temp
#         date_ref_temp_df = pd.DataFrame({"visit_start":visit_start_temp, "start_date":start_date_temp,"end_date":end_date_temp})
              
#         print(date_ref_temp_df)
        
        if date_ref_df.empty == True:
            date_ref_df = date_ref_temp_df 
        else:
            date_ref_df = pd.concat([date_ref_df, date_ref_temp_df]) 
            
        date_ref_temp_df = pd.DataFrame() #date_ref_temp_df.iloc[0:0]
        
    return date_ref_df
        

#         print(start_date_temp)
#         print(end_date_temp)
    

In [11]:
date_ref_df = date_ref_table_gen ('2022-06-26', 28, 14) #2021-10-13

In [155]:
date_ref_df

,visit_start,start_date,end_date
0,2022-05-29,2022-06-26,2022-07-23
1,2022-06-26,2022-07-24,2022-08-20
2,2022-07-24,2022-08-21,2022-09-17
3,2022-08-21,2022-09-18,2022-10-15
4,2022-09-18,2022-10-16,2022-11-12
5,2022-10-16,2022-11-13,2022-12-10
6,2022-11-13,2022-12-11,2023-01-07
7,2022-12-11,2023-01-08,2023-02-04
8,2023-01-08,2023-02-05,2023-03-04
9,2023-02-05,2023-03-05,2023-04-01


In [198]:
iq.upload(date_ref_df, 'sandbox_db.dianedou.ott_incrementality_vprod_date_ref_df')

INFO:snowflake.connector.connection:Snowflake Connector for Python Version: 2.7.7, Python Version: 3.7.13, Platform: Darwin-21.6.0-x86_64-i386-64bit
INFO:snowflake.connector.connection:This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
INFO:snowflake.connector.cursor:query: [USE sandbox_db]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [USE SCHEMA dianedou]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [CREATE OR REPLACE STAGE instaquery_data_stage_08e0d002b80a45a9b43ea512141d983e F...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [PUT file:///private/var/folders/c5/xzbsv5591kqbgc2dqqy9xmv40000gp/T/tmpqayvrr32 ...]
INFO:snowflake.connector.cursor:query execution do

True

In [13]:
params = {}
# params['user_base_table'] = user_base_table
# params['reporting_date'] = str(reporting_date)

In [14]:
def df_user_base_query(start_date_temp, end_date_temp, visit_start_temp):
    base_table_query_temp = f"""set start_date = '{start_date_temp}'; set end_date = '{end_date_temp}'; set visit_start = '{visit_start_temp}'; """
    SnowflakeDb.singleton().execute_sql(base_table_query_temp)

    with open('OTT_feature_gen_new_user.sql', 'r') as f:
        sql = f.read()
        query_feature = jinja2.Template(sql).render(params=params)
        SnowflakeDb.singleton().execute_sql(query_feature)
    return 

In [19]:
for row in range(0, date_ref_df.shape[0]): #date_ref_df.shape[0]
    print(row)
    cohort_temp = date_ref_df.iloc[row, ]
    print(cohort_temp)
    
    start_date_temp = cohort_temp['start_date']
    end_date_temp  = cohort_temp['end_date']
    visit_start_temp = cohort_temp['visit_start']
    
    df_user_base_query(start_date_temp, end_date_temp, visit_start_temp)
    
#     previous_df = SnowflakeDb.singleton().to_dataframe("select * from sandbox_db.dianedou.OTT_matching_attributes_vprod")
#     previous_df.columns = previous_df.columns.str.lower()
#     previous_df.cohort = pd.to_datetime(previous_df.cohort)
#     if pd.Timestamp(start_date_temp) not in list(set(previous_df.cohort.to_list())):
#         SnowflakeDb.singleton().execute_sql('CREATE OR REPLACE table sandbox_db.dianedou.OTT_matching_attributes_vprod as (select * from sandbox_db.dianedou.OTT_matching_attributes_vprod union all select * from sandbox_db.dianedou.OTT_matching_attributes_vprod_temp)')
    try:
        previous_df = SnowflakeDb.singleton().to_dataframe("select * from sandbox_db.dianedou.OTT_new_user_matching_attributes_vprod")
        previous_df.columns = previous_df.columns.str.lower()
        previous_df.cohort = pd.to_datetime(previous_df.cohort)
        if pd.Timestamp(start_date_temp) not in list(set(previous_df.cohort.to_list())):
            SnowflakeDb.singleton().execute_sql('CREATE OR REPLACE table sandbox_db.dianedou.OTT_new_user_matching_attributes_vprod as (select * from sandbox_db.dianedou.OTT_new_user_matching_attributes_vprod union all select * from sandbox_db.dianedou.OTT_new_user_matching_attributes_vprod_temp)')
    except:
        SnowflakeDb.singleton().execute_sql('CREATE OR REPLACE table sandbox_db.dianedou.OTT_new_user_matching_attributes_vprod as (select * from sandbox_db.dianedou.OTT_new_user_matching_attributes_vprod_temp)')


INFO:root:>>>> [query-start] SnowflakeDb: set start_date = '2022-09-18';...
  File "/Users/dianedou/carrot/analytics/piper/piper/trace_utils.py", line 51, in __wrapper
    return fn(*args, **kwargs)
>> SQL: set start_date = '2022-09-18';
<<
INFO:snowflake.connector.cursor:query: [set start_date = '2022-09-18';]


3
visit_start    2022-08-21
start_date     2022-09-18
end_date       2022-10-15
Name: 3, dtype: object


INFO:snowflake.connector.cursor:query execution done
INFO:root:<<<< [query-done] SnowflakeDb: duration=0:00:00.540680 query_id=01ae33e2-0406-8b19-0004-1509869131cb
INFO:root:>>>> [query-start] SnowflakeDb: set end_date = '2022-10-15';...
  File "/Users/dianedou/carrot/analytics/piper/piper/trace_utils.py", line 51, in __wrapper
    return fn(*args, **kwargs)
>> SQL: set end_date = '2022-10-15';
<<
INFO:snowflake.connector.cursor:query: [set end_date = '2022-10-15';]
INFO:snowflake.connector.cursor:query execution done
INFO:root:<<<< [query-done] SnowflakeDb: duration=0:00:00.127926 query_id=01ae33e2-0406-8b88-0004-15098691402b
INFO:root:>>>> [query-start] SnowflakeDb: set visit_start = '2022-08-21'...
  File "/Users/dianedou/carrot/analytics/piper/piper/trace_utils.py", line 51, in __wrapper
    return fn(*args, **kwargs)
>> SQL: set visit_start = '2022-08-21';
<<
INFO:snowflake.connector.cursor:query: [set visit_start = '2022-08-21';]
INFO:snowflake.connector.cursor:query execution do

INFO:snowflake.connector.cursor:query: [create or replace table sandbox_db.dianedou.base_features_OTT_vprod_temp as  WIT...]
INFO:snowflake.connector.cursor:query execution done
INFO:root:<<<< [query-done] SnowflakeDb: duration=0:01:55.764873 query_id=01ae33e4-0406-8b52-0004-15098691baeb
INFO:root:>>>> [query-start] SnowflakeDb: create or replace table sandbo...
  File "/Users/dianedou/carrot/analytics/piper/piper/trace_utils.py", line 51, in __wrapper
    return fn(*args, **kwargs)
>> SQL: create or replace table sandbox_db.dianedou.express_past_express_status_OTT_vprod_temp as

  SELECT
    base.user_id,
    base.created_at_pt,
    max(cast(    (subscription_type = 'paid') and (term_length = 'year') as int) ) as past_annual,
    max(cast(   (subscription_type = 'paid') and (term_length in ('month','three months','six months')) as int) ) as past_monthly,
    max(cast(   (subscription_type = 'free')  as int) ) as past_trial
    from instadata.rds_data.subscriptions s
    join table($us

INFO:snowflake.connector.cursor:query: [CREATE OR REPLACE table sandbox_db.dianedou.itemattributes_OTT_vprod_temp AS SEL...]
INFO:snowflake.connector.cursor:query execution done
INFO:root:<<<< [query-done] SnowflakeDb: duration=0:00:18.613417 query_id=01ae33e8-0406-8b8a-0004-150986947f17
INFO:root:>>>> [query-start] SnowflakeDb: CREATE OR REPLACE table sandbo...
  File "/Users/dianedou/carrot/analytics/piper/piper/trace_utils.py", line 51, in __wrapper
    return fn(*args, **kwargs)
>> SQL: CREATE OR REPLACE table sandbox_db.dianedou.OTT_zip_median_income as

select mut.*
, median_income
from table($user_base_table) mut
JOIN INSTADATA.ECLIPSE.DIM_USER du on mut.user_id = du.user_id
JOIN instadata.analysts.demographis_income_by_zip_code income on du.active_zip_code = income.zip_code
;
<<
INFO:snowflake.connector.cursor:query: [CREATE OR REPLACE table sandbox_db.dianedou.OTT_zip_median_income as  select mut...]
INFO:snowflake.connector.cursor:query execution done
INFO:root:<<<< [query-do

...

INFO:snowflake.connector.cursor:query: [set start_date = '2023-06-25';]
INFO:snowflake.connector.cursor:query execution done
INFO:root:<<<< [query-done] SnowflakeDb: duration=0:00:00.098128 query_id=01ae342c-0406-8b0d-0004-150986bf9a1f
INFO:root:>>>> [query-start] SnowflakeDb: set end_date = '2023-07-22';...
  File "/Users/dianedou/carrot/analytics/piper/piper/trace_utils.py", line 51, in __wrapper
    return fn(*args, **kwargs)
>> SQL: set end_date = '2023-07-22';
<<
INFO:snowflake.connector.cursor:query: [set end_date = '2023-07-22';]
INFO:snowflake.connector.cursor:query execution done


13
visit_start    2023-05-28
start_date     2023-06-25
end_date       2023-07-22
Name: 13, dtype: object


INFO:root:<<<< [query-done] SnowflakeDb: duration=0:00:00.154264 query_id=01ae342c-0406-8b8a-0004-150986bff637
INFO:root:>>>> [query-start] SnowflakeDb: set visit_start = '2023-05-28'...
  File "/Users/dianedou/carrot/analytics/piper/piper/trace_utils.py", line 51, in __wrapper
    return fn(*args, **kwargs)
>> SQL: set visit_start = '2023-05-28';
<<
INFO:snowflake.connector.cursor:query: [set visit_start = '2023-05-28';]
INFO:snowflake.connector.cursor:query execution done
INFO:root:<<<< [query-done] SnowflakeDb: duration=0:00:00.154037 query_id=01ae342c-0406-8b16-0004-150986c00447
INFO:root:>>>> [query-start] SnowflakeDb: USE ROLE IC_DS_ROLE;...
  File "/Users/dianedou/carrot/analytics/piper/piper/trace_utils.py", line 51, in __wrapper
    return fn(*args, **kwargs)
>> SQL: USE ROLE IC_DS_ROLE;
<<
INFO:snowflake.connector.cursor:query: [USE ROLE IC_DS_ROLE;]
INFO:snowflake.connector.cursor:query execution done
INFO:root:<<<< [query-done] SnowflakeDb: duration=0:00:00.089203 query_id=

INFO:snowflake.connector.cursor:query: [create or replace table sandbox_db.dianedou.base_features_OTT_vprod_temp as  WIT...]
INFO:snowflake.connector.cursor:query execution done
INFO:root:<<<< [query-done] SnowflakeDb: duration=0:01:47.448963 query_id=01ae342c-0406-8b0d-0004-150986c082fb
INFO:root:>>>> [query-start] SnowflakeDb: create or replace table sandbo...
  File "/Users/dianedou/carrot/analytics/piper/piper/trace_utils.py", line 51, in __wrapper
    return fn(*args, **kwargs)
>> SQL: create or replace table sandbox_db.dianedou.express_past_express_status_OTT_vprod_temp as

  SELECT
    base.user_id,
    base.created_at_pt,
    max(cast(    (subscription_type = 'paid') and (term_length = 'year') as int) ) as past_annual,
    max(cast(   (subscription_type = 'paid') and (term_length in ('month','three months','six months')) as int) ) as past_monthly,
    max(cast(   (subscription_type = 'free')  as int) ) as past_trial
    from instadata.rds_data.subscriptions s
    join table($us

INFO:snowflake.connector.cursor:query: [CREATE OR REPLACE table sandbox_db.dianedou.itemattributes_OTT_vprod_temp AS SEL...]
INFO:snowflake.connector.cursor:query execution done
INFO:root:<<<< [query-done] SnowflakeDb: duration=0:00:22.363023 query_id=01ae3430-0406-8b88-0004-150986c2c1df
INFO:root:>>>> [query-start] SnowflakeDb: CREATE OR REPLACE table sandbo...
  File "/Users/dianedou/carrot/analytics/piper/piper/trace_utils.py", line 51, in __wrapper
    return fn(*args, **kwargs)
>> SQL: CREATE OR REPLACE table sandbox_db.dianedou.OTT_zip_median_income as

select mut.*
, median_income
from table($user_base_table) mut
JOIN INSTADATA.ECLIPSE.DIM_USER du on mut.user_id = du.user_id
JOIN instadata.analysts.demographis_income_by_zip_code income on du.active_zip_code = income.zip_code
;
<<
INFO:snowflake.connector.cursor:query: [CREATE OR REPLACE table sandbox_db.dianedou.OTT_zip_median_income as  select mut...]
INFO:snowflake.connector.cursor:query execution done
INFO:root:<<<< [query-do

In [ ]:
#### generate outcome ####

In [160]:
def df_outcome_query(cohort_temp):
    base_table_query_temp = f"""set cohort_temp = '{cohort_temp}'; set user_base_table = 'sandbox_db.dianedou.OTT_new_user_matching_attributes_vprod';"""
    
    SnowflakeDb.singleton().execute_sql(base_table_query_temp)

    with open('OTT_outcome_gen_new_user.sql', 'r') as f:
        sql = f.read()
        query_feature = jinja2.Template(sql).render(params=params)
        SnowflakeDb.singleton().execute_sql(query_feature)
    return 

In [158]:
for row in range(0, date_ref_df.shape[0]): 

    cohort_temp = date_ref_df.loc[row, 'start_date']
    print(cohort_temp)

2022-06-26
2022-07-24
2022-08-21
2022-09-18
2022-10-16
2022-11-13
2022-12-11
2023-01-08
2023-02-05
2023-03-05
2023-04-02
2023-04-30
2023-05-28
2023-06-25


In [168]:
start_date_temp

'2023-06-25'

In [167]:
for row in range(0, date_ref_df.shape[0]): 

    cohort_temp = date_ref_df.loc[row, 'start_date']
    print(cohort_temp)

    df_outcome_query(cohort_temp)
    
    try:
        previous_df = SnowflakeDb.singleton().to_dataframe("select * from sandbox_db.dianedou.OTT_new_user_reporting_metrics_28d_vprod")
        previous_df.columns = previous_df.columns.str.lower()
        previous_df.cohort = pd.to_datetime(previous_df.cohort)
        if pd.Timestamp(cohort_temp) not in list(set(previous_df.cohort.to_list())):
            SnowflakeDb.singleton().execute_sql('CREATE OR REPLACE table sandbox_db.dianedou.OTT_new_user_reporting_metrics_28d_vprod as (select * from sandbox_db.dianedou.OTT_new_user_reporting_metrics_28d_vprod union all select * from sandbox_db.dianedou.OTT_new_user_reporting_metrics_28d_vprod_temp)')
            SnowflakeDb.singleton().execute_sql('CREATE OR REPLACE table sandbox_db.dianedou.OTT_new_user_reporting_metrics_orders_28d_vprod as (select * from sandbox_db.dianedou.OTT_new_user_reporting_metrics_orders_28d_vprod union all select * from sandbox_db.dianedou.OTT_new_user_reporting_metrics_orders_28d_vprod_temp)')
    except:
        SnowflakeDb.singleton().execute_sql('CREATE OR REPLACE table sandbox_db.dianedou.OTT_new_user_reporting_metrics_28d_vprod as (select * from sandbox_db.dianedou.OTT_new_user_reporting_metrics_28d_vprod_temp)')
        SnowflakeDb.singleton().execute_sql('CREATE OR REPLACE table sandbox_db.dianedou.OTT_new_user_reporting_metrics_orders_28d_vprod as (select * from sandbox_db.dianedou.OTT_new_user_reporting_metrics_orders_28d_vprod_temp)')


INFO:root:>>>> [query-start] SnowflakeDb: set cohort_temp = '2022-06-26'...
  File "/Users/dianedou/carrot/analytics/piper/piper/trace_utils.py", line 51, in __wrapper
    return fn(*args, **kwargs)
>> SQL: set cohort_temp = '2022-06-26';
<<
INFO:snowflake.connector.cursor:query: [set cohort_temp = '2022-06-26';]
INFO:snowflake.connector.cursor:query execution done
INFO:root:<<<< [query-done] SnowflakeDb: duration=0:00:00.098253 query_id=01ae5059-0406-8b88-0004-15099959150f
INFO:root:>>>> [query-start] SnowflakeDb: set user_base_table = 'sandbox...
  File "/Users/dianedou/carrot/analytics/piper/piper/trace_utils.py", line 51, in __wrapper
    return fn(*args, **kwargs)
>> SQL: set user_base_table = 'sandbox_db.dianedou.OTT_new_user_matching_attributes_vprod';
<<
INFO:snowflake.connector.cursor:query: [set user_base_table = 'sandbox_db.dianedou.OTT_new_user_matching_attributes_vpro...]


2022-06-26


INFO:snowflake.connector.cursor:query execution done
INFO:root:<<<< [query-done] SnowflakeDb: duration=0:00:00.106445 query_id=01ae5059-0406-8b88-0004-150999591513
INFO:root:>>>> [query-start] SnowflakeDb: //============================...
  File "/Users/dianedou/carrot/analytics/piper/piper/trace_utils.py", line 51, in __wrapper
    return fn(*args, **kwargs)
>> SQL: //================================================================================================================================================
//=========================================================== outcome generation =================================================================
//================================================================================================================================================



CREATE OR REPLACE table sandbox_db.dianedou.OTT_new_user_reporting_metrics_orders_28d_vprod_temp as

(with base as
    (select omni_user_id as user_id,
     o.value:deliveries  as deliv

2022-07-24


INFO:snowflake.connector.cursor:query execution done
INFO:root:<<<< [query-done] SnowflakeDb: duration=0:00:00.237561 query_id=01ae505c-0406-8b19-0004-1509995a8c33
INFO:root:>>>> [query-start] SnowflakeDb: set user_base_table = 'sandbox...
  File "/Users/dianedou/carrot/analytics/piper/piper/trace_utils.py", line 51, in __wrapper
    return fn(*args, **kwargs)
>> SQL: set user_base_table = 'sandbox_db.dianedou.OTT_new_user_matching_attributes_vprod';
<<
INFO:snowflake.connector.cursor:query: [set user_base_table = 'sandbox_db.dianedou.OTT_new_user_matching_attributes_vpro...]
INFO:snowflake.connector.cursor:query execution done
INFO:root:<<<< [query-done] SnowflakeDb: duration=0:00:00.187748 query_id=01ae505c-0406-8b88-0004-1509995b2203
INFO:root:>>>> [query-start] SnowflakeDb: //============================...
  File "/Users/dianedou/carrot/analytics/piper/piper/trace_utils.py", line 51, in __wrapper
    return fn(*args, **kwargs)
>> SQL: //============================================

...

INFO:snowflake.connector.cursor:query execution done
INFO:root:<<<< [query-done] SnowflakeDb: duration=0:00:00.088430 query_id=01ae507e-0406-8b8a-0004-1509996f81b7
INFO:root:>>>> [query-start] SnowflakeDb: //============================...
  File "/Users/dianedou/carrot/analytics/piper/piper/trace_utils.py", line 51, in __wrapper
    return fn(*args, **kwargs)
>> SQL: //================================================================================================================================================
//=========================================================== outcome generation =================================================================
//================================================================================================================================================



CREATE OR REPLACE table sandbox_db.dianedou.OTT_new_user_reporting_metrics_orders_28d_vprod_temp as

(with base as
    (select omni_user_id as user_id,
     o.value:deliveries  as deliv

2023-05-28


INFO:root:>>>> [query-start] SnowflakeDb: //============================...
  File "/Users/dianedou/carrot/analytics/piper/piper/trace_utils.py", line 51, in __wrapper
    return fn(*args, **kwargs)
>> SQL: //================================================================================================================================================
//=========================================================== outcome generation =================================================================
//================================================================================================================================================



CREATE OR REPLACE table sandbox_db.dianedou.OTT_new_user_reporting_metrics_orders_28d_vprod_temp as

(with base as
    (select omni_user_id as user_id,
     o.value:deliveries  as deliveries,
     o.VALUE:order_type as order_type,
     o.value:order_id as order_id,
     date_trunc('day', o.value:order_created_date_time_pt::datetime) as order_creat

2023-06-25


INFO:snowflake.connector.cursor:query execution done
INFO:root:<<<< [query-done] SnowflakeDb: duration=0:00:00.157572 query_id=01ae5083-0406-8b16-0004-150999715f73
INFO:root:>>>> [query-start] SnowflakeDb: //============================...
  File "/Users/dianedou/carrot/analytics/piper/piper/trace_utils.py", line 51, in __wrapper
    return fn(*args, **kwargs)
>> SQL: //================================================================================================================================================
//=========================================================== outcome generation =================================================================
//================================================================================================================================================



CREATE OR REPLACE table sandbox_db.dianedou.OTT_new_user_reporting_metrics_orders_28d_vprod_temp as

(with base as
    (select omni_user_id as user_id,
     o.value:deliveries  as deliv

In [ ]:
#### matching ####

In [34]:
def remove_elements_from_list (list_keep, list_remove):
    for i in list_remove:
        try:
            list_keep.remove(i)
        except:
            print(i + ' does not exist')
    return list_keep

In [122]:
def rename_cols(df):
    df.columns = np.char.replace(np.char.replace(np.char.lower(df.columns.values.astype('str')), ':overall::numeric(10,2)',''), 'fuga.','')


In [125]:
feature = [
 'visits_l7',
 'visits_l28',
 'visits_l91',
 'visits_lifetime',
 'signup_days',
 'past_annual',
 'past_monthly',
 'past_trial',
 'zip_median_income',
 'tot_visit_time',
 'n_platforms_visited',
 'ever_mobile',
 'tot_visit_time_l28',
 'cnt_visits_l28',
 'n_platforms_visited_l28',
 'ever_mobile_l28'
 ]

In [152]:
feature

['visits_l7',
 'visits_l28',
 'visits_l91',
 'visits_lifetime',
 'signup_days',
 'past_annual',
 'past_monthly',
 'past_trial',
 'zip_median_income',
 'tot_visit_time',
 'n_platforms_visited',
 'ever_mobile',
 'tot_visit_time_l28',
 'cnt_visits_l28',
 'n_platforms_visited_l28',
 'ever_mobile_l28']

In [133]:
def matching_cohort (data, variant_var = 'variant'):

    control = data.loc[data[variant_var] == 0, ].reset_index(drop=True)
    test = data.loc[data[variant_var] == 1, ].reset_index(drop=True)

    m = Matcher(test, control, yvar=variant_var, exclude=attributes_to_exclude)
    np.random.seed(20170925)
    m.fit_scores(balance=True, nmodels=10)
    m.predict_scores()
    m.match(method="random", nmatches=1, threshold=0.0005)
    matched_df = m.matched_data.reset_index(drop=True)

    return (matched_df)


In [ ]:
#### productionization matching 

In [154]:
df = iq.query("SELECT * FROM sandbox_db.dianedou.OTT_new_user_matching_attributes_vprod")

INFO:snowflake.connector.connection:Snowflake Connector for Python Version: 2.7.7, Python Version: 3.7.13, Platform: Darwin-21.6.0-x86_64-i386-64bit
INFO:snowflake.connector.connection:This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
INFO:snowflake.connector.cursor:query: [SELECT * FROM sandbox_db.dianedou.OTT_new_user_matching_attributes_vprod]
INFO:snowflake.connector.cursor:query execution done
INFO:instaquery.sql.timer:Query ran in 15.95 seconds
INFO:snowflake.connector.connection:closed
INFO:snowflake.connector.connection:No async queries seem to be running, deleting session


In [138]:
df.shape

(2685691, 83)

In [ ]:
# outcome 
# select * from sandbox_db.dianedou.SEM_reporting_metrics_28d_vprod_temp;
# select * from sandbox_db.dianedou.SEM_reporting_metrics_orders_28d_vprod_temp;

In [139]:
df_intervention = df.copy()
obj_df = df_intervention.select_dtypes(include=['object']).copy()

In [140]:
dummy_var = remove_elements_from_list (obj_df.columns.to_list() , [ 'wk_ref', 'created_at_pt', 'cohort', 'measured_date','activation_warehouse_name'])

In [141]:
X_coded = pd.get_dummies(df_intervention, columns = dummy_var, drop_first = True)
# df_intervention_lite = pd.concat([df_intervention[reporting_metric + variant], X_coded], axis = 1)

In [142]:
rename_cols(X_coded)

In [143]:
X_coded = X_coded[feature]

In [144]:
feature

['visits_l7',
 'visits_l28',
 'visits_l91',
 'visits_lifetime',
 'signup_days',
 'past_annual',
 'past_monthly',
 'past_trial',
 'zip_median_income',
 'tot_visit_time',
 'n_platforms_visited',
 'ever_mobile',
 'tot_visit_time_l28',
 'cnt_visits_l28',
 'n_platforms_visited_l28',
 'ever_mobile_l28']

In [145]:
df_intervention_matching = pd.concat([df_intervention[['user_id', 'variant', 'cohort', 'wk_ref']], X_coded], axis = 1)

In [146]:
attributes_to_exclude = remove_elements_from_list (df_intervention_matching.columns.tolist() , feature)

In [147]:
def matching_cohort (data, variant_var = 'variant'):

    control = data.loc[data[variant_var] == 0, ].reset_index(drop=True)
    test = data.loc[data[variant_var] == 1, ].reset_index(drop=True)
    
    m = Matcher(test, control, yvar=variant_var, exclude=attributes_to_exclude)
    np.random.seed(20170925)
    m.fit_scores(balance=True, nmodels=10)
    m.predict_scores()
    m.match(method="random", nmatches=1, threshold=0.0005)
    matched_df = m.matched_data.reset_index(drop=True)

    return (matched_df)


In [148]:
cohort_list = list(set(df['cohort'].tolist()))

In [149]:
sorted(cohort_list)

['2022-06-26',
 '2022-07-24',
 '2022-08-21',
 '2022-09-18',
 '2022-10-16',
 '2022-11-13',
 '2022-12-11',
 '2023-01-08',
 '2023-02-05',
 '2023-03-05',
 '2023-04-02',
 '2023-04-30',
 '2023-05-28',
 '2023-06-25']

In [151]:
for i in range(1, len(cohort_list)):
    cohort_temp = sorted(cohort_list)[i]
    print(cohort_temp)
    df_intervention_matching_cohort = df_intervention_matching[df_intervention_matching['cohort'] == cohort_temp]
    df_matched = matching_cohort(df_intervention_matching_cohort)
    iq.upload(df_matched, 'sandbox_db.dianedou.OTT_matching_attributes_vprod_matched_temp_new_user')
    
    SnowflakeDb.singleton().execute_sql('CREATE OR REPLACE table sandbox_db.dianedou.OTT_matching_attributes_vprod_matched_new_user as (select * from sandbox_db.dianedou.OTT_matching_attributes_vprod_matched_new_user union all select * from sandbox_db.dianedou.OTT_matching_attributes_vprod_matched_temp_new_user)')
         

2022-07-24
Formula:
variant ~ visits_l7+visits_l28+visits_l91+visits_lifetime+signup_days+past_annual+past_monthly+past_trial+zip_median_income+tot_visit_time+n_platforms_visited+ever_mobile+tot_visit_time_l28+cnt_visits_l28+n_platforms_visited_l28+ever_mobile_l28
n majority: 22969
n minority: 2970
Fitting Models on Balanced Samples: 2\10

/Users/dianedou/.local/share/virtualenvs/piper-L-f8rMXc/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Fitting Models on Balanced Samples: 10\10
Average Accuracy: 68.78%


/Users/dianedou/.local/share/virtualenvs/piper-L-f8rMXc/lib/python3.7/site-packages/statsmodels/genmod/families/links.py:187: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
INFO:snowflake.connector.connection:Snowflake Connector for Python Version: 2.7.7, Python Version: 3.7.13, Platform: Darwin-21.6.0-x86_64-i386-64bit
INFO:snowflake.connector.connection:This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
INFO:snowflake.connector.cursor:query: [USE sandbox_db]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [USE SCHEMA dianedou]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [CREATE OR REPLACE STAGE instaquery_data_stage_342c4c07383440a39ba3012de30938de F...]
INFO:snowflake.connector.cursor:query exe

2022-08-21
Formula:
variant ~ visits_l7+visits_l28+visits_l91+visits_lifetime+signup_days+past_annual+past_monthly+past_trial+zip_median_income+tot_visit_time+n_platforms_visited+ever_mobile+tot_visit_time_l28+cnt_visits_l28+n_platforms_visited_l28+ever_mobile_l28
n majority: 23891
n minority: 3074
Fitting Models on Balanced Samples: 3\10

...

INFO:snowflake.connector.connection:Snowflake Connector for Python Version: 2.7.7, Python Version: 3.7.13, Platform: Darwin-21.6.0-x86_64-i386-64bit
INFO:snowflake.connector.connection:This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
INFO:snowflake.connector.cursor:query: [USE sandbox_db]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [USE SCHEMA dianedou]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [CREATE OR REPLACE STAGE instaquery_data_stage_51f0d0b3264046ac8f60d8307547caa5 F...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [PUT file:///private/var/folders/c5/xzbsv5591kqbgc2dqqy9xmv40000gp/T/tmpoqirpvfl ...]
INFO:snowflake.connector.cursor:query execution do

2023-06-25
Formula:
variant ~ visits_l7+visits_l28+visits_l91+visits_lifetime+signup_days+past_annual+past_monthly+past_trial+zip_median_income+tot_visit_time+n_platforms_visited+ever_mobile+tot_visit_time_l28+cnt_visits_l28+n_platforms_visited_l28+ever_mobile_l28
n majority: 23378
n minority: 3127
Fitting Models on Balanced Samples: 3\10

/Users/dianedou/.local/share/virtualenvs/piper-L-f8rMXc/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Fitting Models on Balanced Samples: 10\10
Average Accuracy: 70.1%


INFO:snowflake.connector.connection:Snowflake Connector for Python Version: 2.7.7, Python Version: 3.7.13, Platform: Darwin-21.6.0-x86_64-i386-64bit
INFO:snowflake.connector.connection:This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
INFO:snowflake.connector.cursor:query: [USE sandbox_db]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [USE SCHEMA dianedou]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [CREATE OR REPLACE STAGE instaquery_data_stage_3afaf5db8f944cd8bd664541e7ef4c16 F...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [PUT file:///private/var/folders/c5/xzbsv5591kqbgc2dqqy9xmv40000gp/T/tmpoijmp753 ...]
INFO:snowflake.connector.cursor:query execution do